# Montar drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Instalar dependencias

In [ ]:
! pip install duckdb pandas
! pip install pyodbc
! pip install ipython

In [ ]:
!apt-get update
!apt-get install -y mdbtools

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 4s (30.8 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [ ]:
import pandas as pd
import datetime
import os
import re
import subprocess
from IPython.display import display
import duckdb

# Configuracion

In [ ]:
# Ruta al archivo .accdb
db_path = '/content/drive/MyDrive/Consulting/Alpha_Espai/POC/SUMAS_Y_SALDOS/'
db_path_INTERMEDIATE = '/content/drive/MyDrive/Consulting/Alpha_Espai/POC/SUMAS_Y_SALDOS/INTERMEDIATE/'
db_path_output = '/content/drive/MyDrive/Consulting/Alpha_Espai/POC/SUMAS_Y_SALDOS/FINAL_RESULT/'

#file = "2322024.accdb"  # Cambiar el nombre del archivo según sea necesario
file = "2302023.accdb"  # Cambiar el nombre del archivo según sea necesario

access_file_path = os.path.join(db_path, file)
file_path = os.path.join(db_path_INTERMEDIATE, file)
file_without_extension = os.path.splitext(file_path)[0]

# Nombre de la tabla que deseas leer
table_name = 'F_APU'  # Cambiar el nombre de la tabla según sea necesario

# Obtener el mes actual
current_month = datetime.datetime.now().month

####
# Parsea nombre fichero para extraer codigo de Franquiciado y año de los datos

# Extraer el código de Franquiciado y el año
last_dot_index = file_path.rfind('.')

if last_dot_index != -1:
    codigo_Franquiciado = file_path[last_dot_index - 7:last_dot_index - 4]
    year = file_path[last_dot_index - 4:last_dot_index]
    print(f"Código de Franquiciado: {codigo_Franquiciado}")
    print(f"Año: {year}")
else:
    print("No se encontró un punto en el nombre del archivo.")

# Formatear el nombre del archivo con el mes actual
output_path = f'{file_without_extension}_{current_month}.csv'
print(f"Ruta de salida: {output_path}")

# Formatear el nombre del fichero en crudo
file_name_without_extension = os.path.splitext(file)[0]
raw_file = f'{db_path_INTERMEDIATE}raw_{file_name_without_extension}_{current_month}.csv'

Código de Franquiciado: 230
Año: 2023
Ruta de salida: /content/drive/MyDrive/Consulting/Alpha_Espai/POC/SUMAS_Y_SALDOS/INTERMEDIATE/2302023_10.csv


# Conecta con Database Access y extrae tabla F_APU en un df

In [ ]:
# Comando para listar las tablas en la base de datos
list_tables_cmd = f'mdb-tables -1 {access_file_path}'
tables = subprocess.check_output(list_tables_cmd, shell=True).decode().split()

# Verificar si la tabla existe
if table_name in tables:
    # Comando para exportar la tabla a un archivo CSV
    export_cmd = f'mdb-export {access_file_path} {table_name}'
    df = pd.read_csv(subprocess.Popen(export_cmd, shell=True, stdout=subprocess.PIPE).stdout)

    # Mostrar el DataFrame utilizando display
    display(df)
else:
    print(f"La tabla '{table_name}' no existe en la base de datos.")

,DIAAPU,FECAPU,ASIAPU,ORDAPU,CUEAPU,IMPAPU,IMEAPU,MONAPU,CONAPU,DOCAPU,...,DEPAPU,SUBAPU,IMAAPU,UCRAPU,FCRAPU,UUMAPU,FUMAPU,GUIDAPU,UKBAPU,TRZAPU
0,1,01/02/23 00:00:00,13,1,4850001,2010.22,2010.22,E,"ASIENTO VENTA DIA ANER ENERGIA, S.L. 02/01/2023",NaN,...,0,0,NaN,0,01/01/00 00:00:00,0,01/01/00 00:00:00,NaN,0,NaN
1,1,01/02/23 00:00:00,13,2,4850002,2.99,2.99,E,"ASIENTO VENTA DIA ANER ENERGIA, S.L. 02/01/2023",NaN,...,0,0,NaN,0,01/01/00 00:00:00,0,01/01/00 00:00:00,NaN,0,NaN
2,1,01/02/23 00:00:00,13,3,5700200,2007.23,2007.23,E,"ASIENTO VENTA DIA ANER ENERGIA, S.L. 02/01/2023",NaN,...,0,0,NaN,0,01/01/00 00:00:00,0,01/01/00 00:00:00,NaN,0,NaN
3,1,01/03/23 00:00:00,27,1,4850001,1846.70,1846.70,E,"ASIENTO VENTA DIA ANER ENERGIA, S.L. 03/01/2023",NaN,...,0,0,NaN,0,01/01/00 00:00:00,0,01/01/00 00:00:00,NaN,0,NaN
4,1,01/03/23 00:00:00,27,2,4850002,1.32,1.32,E,"ASIENTO VENTA DIA ANER ENERGIA, S.L. 03/01/2023",NaN,...,0,0,NaN,0,01/01/00 00:00:00,0,01/01/00 00:00:00,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50232,1,12/31/23 00:00:00,7926,116,5720007,17060.68,17060.68,E,CIERRE DEL EJERCICIO,NaN,...,0,0,NaN,14,08/01/24 00:00:00,14,08/01/24 13:18:25,NaN,0,NaN
50233,1,12/31/23 00:00:00,7926,117,5720013,2157.31,2157.31,E,CIERRE DEL EJERCICIO,NaN,...,0,0,NaN,14,08/01/24 00:00:00,14,08/01/24 13:18:25,NaN,0,NaN
50234,1,12/31/23 00:00:00,7926,118,5720206,142290.11,142290.11,E,CIERRE DEL EJERCICIO,NaN,...,0,0,NaN,14,08/01/24 00:00:00,14,08/01/24 13:18:25,NaN,0,NaN
50235,1,12/31/23 00:00:00,7926,119,5790570,6467.57,6467.57,E,CIERRE DEL EJERCICIO,NaN,...,0,0,NaN,14,08/01/24 00:00:00,14,08/01/24 13:18:25,NaN,0,NaN


# Monta tabla con los datos relevantes

In [ ]:
# Extrae columnas necesarias
query = 'SELECT FECAPU, CUEAPU, CONAPU, IMPAPU, IMEAPU, "D-HAPU", DEPAPU FROM df'
result = duckdb.sql(query)

# Convert to DataFrame
result_df = result.fetchdf()

# Crea nuevas columnas para nombre de fichero
result_df['Mes'] = current_month
result_df['Año'] = year
result_df['Franquiciado'] = codigo_Franquiciado

# Mostrar el DataFrame utilizando display
display(result_df)

,FECAPU,CUEAPU,CONAPU,IMPAPU,IMEAPU,D-HAPU,DEPAPU,Mes,Año,Franquiciado
0,01/02/23 00:00:00,4850001,"ASIENTO VENTA DIA ANER ENERGIA, S.L. 02/01/2023",2010.22,2010.22,H,0,10,2023,230
1,01/02/23 00:00:00,4850002,"ASIENTO VENTA DIA ANER ENERGIA, S.L. 02/01/2023",2.99,2.99,D,0,10,2023,230
2,01/02/23 00:00:00,5700200,"ASIENTO VENTA DIA ANER ENERGIA, S.L. 02/01/2023",2007.23,2007.23,D,0,10,2023,230
3,01/03/23 00:00:00,4850001,"ASIENTO VENTA DIA ANER ENERGIA, S.L. 03/01/2023",1846.70,1846.70,H,0,10,2023,230
4,01/03/23 00:00:00,4850002,"ASIENTO VENTA DIA ANER ENERGIA, S.L. 03/01/2023",1.32,1.32,D,0,10,2023,230
...,...,...,...,...,...,...,...,...,...,...
50232,12/31/23 00:00:00,5720007,CIERRE DEL EJERCICIO,17060.68,17060.68,H,0,10,2023,230
50233,12/31/23 00:00:00,5720013,CIERRE DEL EJERCICIO,2157.31,2157.31,H,0,10,2023,230
50234,12/31/23 00:00:00,5720206,CIERRE DEL EJERCICIO,142290.11,142290.11,H,0,10,2023,230
50235,12/31/23 00:00:00,5790570,CIERRE DEL EJERCICIO,6467.57,6467.57,H,0,10,2023,230


# Elimina filas con codigo DEPAPU igual a 0 (no asignados a una tienda) y ordena por DEPAPAU

In [ ]:
# Combine filtering and sorting into a single SQL query
query = """
SELECT *
FROM result_df
WHERE DEPAPU != 0
ORDER BY CUEAPU;
"""

# Execute the query and get the result as a DataFrame
filtered_sorted_df = duckdb.sql(query).df()

# Display the filtered DataFrame
display(filtered_sorted_df)

,FECAPU,CUEAPU,CONAPU,IMPAPU,IMEAPU,D-HAPU,DEPAPU,Mes,Año,Franquiciado
0,01/01/23 00:00:00,4000110,"FRESCARNEXPESS, S.L S. FRA: 12 - 2022",4546.37,4546.37,D,8181,10,2023,230
1,01/31/23 00:00:00,4770000,"ANER ENERGIA, S.L. VENTA MES 1",6108.32,6108.32,H,8181,10,2023,230
2,01/31/23 00:00:00,4770000,"ANER ENERGIA, S.L. VENTA MES 1",7429.22,7429.22,H,4338,10,2023,230
3,01/31/23 00:00:00,4770000,"ANER ENERGIA, S.L. VENTA MES 1",9755.16,9755.16,H,4897,10,2023,230
4,01/31/23 00:00:00,4770000,"ANER ENERGIA, S.L. VENTA MES 1",8618.48,8618.48,H,5077,10,2023,230
...,...,...,...,...,...,...,...,...,...,...
14627,12/15/23 00:00:00,7590010,CC CARREF: 0721,152.08,152.08,H,5077,10,2023,230
14628,12/15/23 00:00:00,7590010,CC CARREF: 0721,152.08,152.08,H,5379,10,2023,230
14629,12/15/23 00:00:00,7590010,CC CARREF: 0721,152.09,152.09,H,8181,10,2023,230
14630,12/12/23 00:00:00,7590015,CC CARREF: 0709,36500.00,36500.00,H,8181,10,2023,230


# Salva tabla en crudo sin agregar valores con fromato raw_XXX20XX.csv

In [ ]:
# Guardar el DataFrame result en un archivo CSV
#file_name_without_extension = os.path.splitext(file)[0]
#raw_file = f'{db_path}raw_{file_name_without_extension}_{current_month}.csv'

filtered_sorted_df.to_csv(raw_file, index=False)

# Sumar IMEAPU por coincidencia de codigo CUEAPU cuando sean iguales valores de D-HAPU y DEPAPU


In [ ]:
query = """
SELECT
    CUEAPU,
    "D-HAPU",
    DEPAPU,
    Año,
    Mes,
    Franquiciado,
    SUM(IMEAPU) as total_IMEAPU,
    COUNT(*) as cantidad_registros
FROM filtered_sorted_df
GROUP BY CUEAPU, "D-HAPU", DEPAPU, Año, Mes, Franquiciado;
"""

final_df = duckdb.sql(query).df()
display(final_df)

,CUEAPU,D-HAPU,DEPAPU,Año,Mes,Franquiciado,total_IMEAPU,cantidad_registros
0,4000110,D,8181,2023,10,230,4546.37,1
1,4770000,H,4338,2023,10,230,104243.79,12
2,4770000,H,4897,2023,10,230,134406.25,12
3,4770000,H,5077,2023,10,230,112761.51,12
4,4770000,H,5379,2023,10,230,85825.31,12
...,...,...,...,...,...,...,...,...
429,7590006,H,4897,2023,10,230,6195.98,4
430,7590006,H,5077,2023,10,230,2250.00,2
431,7590010,H,5379,2023,10,230,1326.19,16
432,7590015,H,8181,2023,10,230,36500.00,1


# Incorporar info Franquiciado por codigo DEAPU proveniente de fichero Masterfile_CODIGOS.csv

In [ ]:
# First, read the CSV file into a table in DuckDB
query_import = f"""
CREATE OR REPLACE TABLE codigos AS
SELECT * FROM read_csv_auto('{db_path}/Masterfile_CODIGOS.csv');
"""
duckdb.sql(query_import)

# Now, perform the JOIN and create the final table
query_join = """
CREATE OR REPLACE TABLE final_table AS
SELECT
    f.*,  -- all fields from final_df
    c.SOCIEDAD,
    c.CIF,
    c.ATICA,
    c.Provincia,
    c.TIENDA,
    c."Nº Tiendas",
    c.sociedad_2,

FROM final_df f
LEFT JOIN codigos c ON f.DEPAPU = c.EST;  -- changed DEPAPU to EST
"""
duckdb.sql(query_join)

# Visualize the result
final_df = duckdb.sql("SELECT * FROM final_table").df()
display(final_df)

,CUEAPU,D-HAPU,DEPAPU,Año,Mes,Franquiciado,total_IMEAPU,cantidad_registros,SOCIEDAD,CIF,ATICA,Provincia,TIENDA,Nº Tiendas,sociedad_2
0,4000110,D,8181,2023,10,230,4546.37,1,"ANER ENERGIA, S.L.",B39749379,00RG,Santander,Lierganes,NaN,"ANER ENERGIA, S.L."
1,4770000,H,4338,2023,10,230,104243.79,12,"ANER ENERGIA, S.L.",B39749379,4261,Santander,"Avda. de los Castros, 79",NaN,"ANER ENERGIA, S.L."
2,4770000,H,4897,2023,10,230,134406.25,12,"ANER ENERGIA, S.L.",B39749379,5008,Santander,Calle de la Ronda 32,NaN,"ANER ENERGIA, S.L."
3,4770000,H,5077,2023,10,230,112761.51,12,"ANER ENERGIA, S.L.",B39749379,5248,Santander,"Paseo General Davila, 306",NaN,"ANER ENERGIA, S.L."
4,4770000,H,5379,2023,10,230,85825.31,12,"ANER ENERGIA, S.L.",B39749379,5731,Santander,"Calle Marqués de la Hermida, 14,",NaN,"ANER ENERGIA, S.L."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,7590006,H,5077,2023,10,230,2250.00,2,"ANER ENERGIA, S.L.",B39749379,5248,Santander,"Paseo General Davila, 306",NaN,"ANER ENERGIA, S.L."
430,7590010,H,5379,2023,10,230,1326.19,16,"ANER ENERGIA, S.L.",B39749379,5731,Santander,"Calle Marqués de la Hermida, 14,",NaN,"ANER ENERGIA, S.L."
431,7590015,H,8181,2023,10,230,36500.00,1,"ANER ENERGIA, S.L.",B39749379,00RG,Santander,Lierganes,NaN,"ANER ENERGIA, S.L."
432,7590015,H,4338,2023,10,230,24000.00,1,"ANER ENERGIA, S.L.",B39749379,4261,Santander,"Avda. de los Castros, 79",NaN,"ANER ENERGIA, S.L."


# Salvar fichero Franquiciado (puede incluir más de una tienda)

In [ ]:
# Guardar el DataFrame result en un archivo CSV
#display(final_df)
display (output_path)
final_df.to_csv(output_path, index=False)

'/content/drive/MyDrive/Consulting/Alpha_Espai/POC/SUMAS_Y_SALDOS/INTERMEDIATE/2302023_10.csv'

# Desagrega el fichero final por tienda (columna DEPAPU) y registra en directorio FINAL_RESULT con nombre: **Tienda_DEPAPU_año_mes.csv**

In [ ]:
# Function to clean the TIENDA value
def clean_tienda_value(tienda_value):
    # Ensure the value is a string
    if not isinstance(tienda_value, str):
        return str(tienda_value)

    # Use regex to remove any number at the end of the string, optionally preceded by a comma
    cleaned_value = re.sub(r',?\s*\d+(,\s*)?$', '', tienda_value)
    return cleaned_value

# Loop through unique DEPAPU values
for depapu_value in final_df['DEPAPU'].unique():
    # Filter the DataFrame based on the current DEPAPU value
    df_filtered = final_df[final_df['DEPAPU'] == depapu_value]

    # Loop through unique TIENDA values within the filtered DataFrame
    for tienda_value in df_filtered['TIENDA'].unique():
        # Clean the TIENDA value
        cleaned_tienda_value = clean_tienda_value(tienda_value)

        # Filter the DataFrame further based on the current TIENDA value
        df_tienda = df_filtered[df_filtered['TIENDA'] == tienda_value]

        # Check if the DataFrame is not empty
        if not df_tienda.empty:
            # Extract year and month values (assuming you have those columns)
            year = df_tienda['Año'].iloc[0]  # Assuming year is consistent for each TIENDA
            mes = df_tienda['Mes'].iloc[0]  # Assuming month is consistent for each TIENDA

            # Construct the filename
            filename = f"{db_path_output}{cleaned_tienda_value}_{depapu_value}_{year}_{mes}.csv"
            #print (filename)

            # Save the filtered DataFrame to a CSV file
            df_tienda.to_csv(filename, index=False)

# Print a message when the process is done
print("Datos separados en archivos CSV.")

Datos separados en archivos CSV.
